In [ ]:
pip install sklearn

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import model_selection
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

In [ ]:
ds = pd.read_csv('bdata_dummies.csv')

In [ ]:
ds

,book_authors,book_title,book_desc,book_rating,book_rating_count,book_review_count,genres,anger,fear,happiness,sadness,surprise,emotion_index,16th Century,18th Century,19th Century,20th Century,21st Century,Abandoned,Abuse,Academic,Action,Adoption,Adult,Adult Fiction,Adventure,Africa,African American,African American Literature,Aliens,Alternate History,Amazon,American,American Civil War,American History,American Revolution,Americana,Amish,Ancient,Ancient History,...,Transgender,Travel,True Crime,True Story,Tudor Period,Turkish,Turkish Literature,Ukraine,Unfinished,Urban,Urban Fantasy,Vampires,Vegan,Vegetarian,Vegetarianism,Victorian,Video Games,War,Warfare,Weird Fiction,Werewolves,Western Africa,Western Romance,Westerns,Wildlife,Witchcraft,Witches,Wolves,Womens,Womens Fiction,World History,World War I,World War II,Writing,Young Adult,Young Adult Contemporary,Young Adult Fantasy,Young Adult Paranormal,Young Adult Romance,Zombies
0,Suzanne Collins,The Hunger Games,Winning will make you famous. Losing means cer...,4.33,5519135,160706,Young Adult|Fiction|Science Fiction|Dystopia|F...,0.07,0.48,0.07,0.30,0.07,0.198,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,J.K. Rowling,Harry Potter and the Order of the Phoenix,There is a door at the end of a silent corrido...,4.48,2041594,33264,Fantasy|Young Adult|Fiction,0.04,0.59,0.07,0.11,0.19,0.200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,Harper Lee,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,4.27,3745197,79450,Classics|Fiction|Historical|Historical Fiction...,0.00,0.30,0.11,0.30,0.30,0.202,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Stephenie Meyer,Twilight,About three things I was absolutely positive.F...,3.58,4281268,97991,Young Adult|Fantasy|Romance|Paranormal|Vampire...,0.10,0.20,0.20,0.20,0.30,0.200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,Markus Zusak,The Book Thief,Trying to make sense of the horrors of World W...,4.36,1485632,100821,Historical|Historical Fiction|Fiction|Young Adult,0.12,0.12,0.00,0.12,0.62,0.196,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4746,Steve Kluger,My Most Excellent Year,Dear Anthony:I appreciate your recent interest...,4.04,7279,1072,Young Adult|Fiction|Romance|Lgbt|Contemporary|...,0.08,0.23,0.38,0.08,0.23,0.200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4747,Kimberly Derting,Desires of the Dead,The missing dead call to Violet. They want to ...,4.06,22320,1605,Mystery|Romance|Fantasy|Young Adult|Fantasy|Su...,0.00,0.45,0.07,0.24,0.24,0.200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4748,Eva Ibbotson,The Dragonfly Pool,Tally Hamilton is furious to hear she is being...,4.00,3799,418,Historical|Historical Fiction|Young Adult|Chil...,0.04,0.46,0.19,0.19,0.12,0.200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4749,Fábio Moon,Daytripper,What are the most important days of your life?...,4.30,21965,2029,Sequential Art|Graphic Novels|Sequential Art|C...,0.03,0.32,0

In [ ]:
cols = ds.columns
columns = cols.drop(labels=['book_title', 'book_authors', 'book_desc', 'book_title', 'genres', 'emotion_index'])

In [ ]:
columns

Index(['book_rating', 'book_rating_count', 'book_review_count', 'anger',
       'fear', 'happiness', 'sadness', 'surprise', '16th Century',
       '18th Century',
       ...
       'World History', 'World War I', 'World War II', 'Writing',
       'Young Adult', 'Young Adult Contemporary', 'Young Adult Fantasy',
       'Young Adult Paranormal', 'Young Adult Romance', 'Zombies'],
      dtype='object', length=437)

In [ ]:
#X = ds[[c for c in columns]].values.reshape(-1, 1)
X = ds[[c for c in columns]]
X_train = X[0:3325]
X_test = X[3325:]
#y = ds['book_rating'].values.reshape(-1, 1)
y = ds['book_rating']
y_train = y[0:3325]
y_test = y[3325:]

In [ ]:
X_test.shape

(1426, 437)

In [ ]:
y_test.shape

(1426,)

In [ ]:
md_lr = LinearRegression()
md_lr.fit(X_train, y_train)
pred_lr = md_lr.predict(X_test)
print('linear regression')
print(np.sqrt(mean_squared_error(y_test, pred_lr)))
print(r2_score(y_test, pred_lr))
#print(md_lr.score(X_test, y_test))

linear regression
4.510917530527849e-11
1.0


In [ ]:
md_dt = DecisionTreeRegressor(max_depth=8, min_samples_leaf=0.13, random_state=3)
md_dt.fit(X_train, y_train)
pred_dt = md_dt.predict(X_test)
print('decision tree')
print(np.sqrt(mean_squared_error(y_test, pred_dt)))
print(r2_score(y_test, pred_dt))

decision tree
0.08012814115632104
0.8969821692230014


**K-Fold**

In [ ]:
kf_res = {'1_model': [], '2_fit_time': [], '3_score_time': [], '4_r2': [], '5_mse': []}

*Linear Regression*

In [ ]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)
mdl_lr = LinearRegression().fit(X, y)
scores_lr = cross_validate(mdl_lr, X, y, scoring=('r2', 'neg_mean_squared_error'), cv=cv, n_jobs=-1)

In [ ]:
scores_lr

{'fit_time': array([0.28955889, 0.31321907, 0.31049776, 0.2985518 , 0.30283713,
        0.29665661, 0.29739904, 0.31182504, 0.28971601, 0.25400901]),
 'score_time': array([0.0066967 , 0.00629854, 0.00431919, 0.00433683, 0.00410986,
        0.00450253, 0.00561929, 0.00424647, 0.00423908, 0.00267601]),
 'test_neg_mean_squared_error': array([-1.25326767e-20, -3.85255830e-21, -1.87403842e-20, -1.43116117e-20,
        -7.36749627e-21, -3.99081720e-21, -2.11335696e-21, -1.08709880e-20,
        -4.33572948e-20, -5.81346206e-21]),
 'test_r2': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

In [ ]:
kf_res['1_model'].append('linear regression')
kf_res['2_fit_time'].append(np.mean(scores_lr['fit_time']))
kf_res['3_score_time'].append(np.mean(scores_lr['score_time']))
kf_res['4_r2'].append(np.mean(scores_lr['test_r2']))
kf_res['5_mse'].append(np.abs(np.mean(scores_lr['test_neg_mean_squared_error'])))

*Ridge Regression*

In [ ]:
mdl_rd_cv = RidgeCV(alphas=[0.1, 1, 10, 50, 100, 900, 1e3, 1e6], store_cv_values=True).fit(X, y)

In [ ]:
# the best alpha
rd_alpha = mdl_rd_cv.alpha_
rd_alpha

0.1

In [ ]:
#mdl_rd = RidgeCV(alphas=[0.1, 1, 10, 100, 1e3, 1e6], store_cv_values=True).fit(X, y)
mdl_rd = Ridge(alpha=rd_alpha).fit(X, y)
scores_rd = cross_validate(mdl_rd, X, y, scoring=('r2', 'neg_mean_squared_error'), cv=cv, n_jobs=-1)

In [ ]:
scores_rd

{'fit_time': array([0.13519645, 0.13208437, 0.1186192 , 0.09454131, 0.09329486,
        0.10153365, 0.09325838, 0.09833169, 0.11738968, 0.10252285]),
 'score_time': array([0.00455403, 0.00438714, 0.00477147, 0.00440335, 0.0041647 ,
        0.00388026, 0.00507927, 0.00610209, 0.00440764, 0.00327635]),
 'test_neg_mean_squared_error': array([-1.52262392e-08, -1.52152787e-08, -1.72318964e-08, -1.62162784e-08,
        -1.74219190e-08, -1.59850251e-08, -1.24624226e-08, -1.79850364e-08,
        -1.44511864e-08, -1.73692358e-08]),
 'test_r2': array([0.99999972, 0.99999972, 0.9999997 , 0.99999972, 0.9999997 ,
        0.99999973, 0.99999975, 0.99999971, 0.99999974, 0.99999971])}

In [ ]:
kf_res['1_model'].append('ridge')
kf_res['2_fit_time'].append(np.mean(scores_rd['fit_time']))
kf_res['3_score_time'].append(np.mean(scores_rd['score_time']))
kf_res['4_r2'].append(np.mean(scores_rd['test_r2']))
kf_res['5_mse'].append(np.abs(np.mean(scores_rd['test_neg_mean_squared_error'])))

*Lasso Regression*

In [ ]:
mdl_ls_cv = LassoCV(cv=cv, n_jobs=-1).fit(X, y)

In [ ]:
# the best alpha
ls_alpha = mdl_ls_cv.alpha_
ls_alpha

918.2567991787705

In [ ]:
mdl_ls = Lasso(alpha=ls_alpha).fit(X, y)
scores_ls = cross_validate(mdl_ls, X, y, scoring=('r2', 'neg_mean_squared_error'), cv=cv, n_jobs=-1)

In [ ]:
scores_ls

{'fit_time': array([0.09418392, 0.05041838, 0.04099059, 0.04292178, 0.04202318,
        0.04360843, 0.04336691, 0.03935933, 0.04194689, 0.0390954 ]),
 'score_time': array([0.00467992, 0.00415468, 0.00451255, 0.00477052, 0.00427675,
        0.00471854, 0.00438213, 0.0047009 , 0.00485539, 0.00409341]),
 'test_neg_mean_squared_error': array([-0.05531765, -0.05431303, -0.05767356, -0.05815677, -0.05777717,
        -0.0589477 , -0.05006096, -0.06298616, -0.05596416, -0.05934102]),
 'test_r2': array([-0.00180721, -0.00026924, -0.00192853, -0.00049561, -0.00037415,
        -0.00197188, -0.0008487 , -0.00465767, -0.00221428, -0.00302162])}

In [ ]:
kf_res['1_model'].append('lasso')
kf_res['2_fit_time'].append(np.mean(scores_ls['fit_time']))
kf_res['3_score_time'].append(np.mean(scores_ls['score_time']))
kf_res['4_r2'].append(np.mean(scores_ls['test_r2']))
kf_res['5_mse'].append(np.abs(np.mean(scores_ls['test_neg_mean_squared_error'])))

In [ ]:
kf_res

{'1_model': ['linear regression', 'ridge', 'lasso'],
 '2_fit_time': [0.2964270353317261, 0.10867724418640137, 0.047791481018066406],
 '3_score_time': [0.0047044515609741214,
  0.004502630233764649,
  0.004514479637145996],
 '4_r2': [1.0, 0.9999997203937573, -0.0017588871356867264],
 '5_mse': [1.229506461893756e-20, 1.595645178552409e-08, 0.05705381678134205]}

*Elastic Net Regression*

In [ ]:
mdl_en_cv = ElasticNetCV(l1_ratio=[1, 0.5, 0.2, 0.3, 0.4, 0.01], n_alphas=100, cv=cv, n_jobs=-1).fit(X, y)

In [ ]:
en_alpha = mdl_en_cv.alpha_
en_alpha

91825.67991787705

In [ ]:
en_l1 = mdl_en_cv.l1_ratio_
en_l1

0.01

In [ ]:
mdl_en = ElasticNet(alpha=en_alpha, l1_ratio=en_l1).fit(X, y)
scores_en = cross_validate(mdl_en, X, y, scoring=('r2', 'neg_mean_squared_error'), cv=cv, n_jobs=-1)

In [ ]:
scores_en

{'fit_time': array([0.04680943, 0.08638549, 0.04306602, 0.04565716, 0.05068088,
        0.05280256, 0.04820395, 0.04313946, 0.0466671 , 0.03237224]),
 'score_time': array([0.00425744, 0.00476861, 0.00435972, 0.00507736, 0.00409865,
        0.00533581, 0.00497794, 0.00489378, 0.00461292, 0.00272894]),
 'test_neg_mean_squared_error': array([-0.05531765, -0.05431303, -0.05767356, -0.05815677, -0.05777717,
        -0.0589477 , -0.05006096, -0.06298616, -0.05596416, -0.05934102]),
 'test_r2': array([-0.00180721, -0.00026924, -0.00192853, -0.00049561, -0.00037415,
        -0.00197188, -0.0008487 , -0.00465767, -0.00221428, -0.00302162])}

In [ ]:
kf_res['1_model'].append('elastic_net')
kf_res['2_fit_time'].append(np.mean(scores_en['fit_time']))
kf_res['3_score_time'].append(np.mean(scores_en['score_time']))
kf_res['4_r2'].append(np.mean(scores_en['test_r2']))
kf_res['5_mse'].append(np.abs(np.mean(scores_en['test_neg_mean_squared_error'])))

*Decision Tree Regressor*

In [ ]:
from sklearn.metrics import make_scorer
scorer = make_scorer(mean_squared_error, greater_is_better=False)
pipe_tree = make_pipeline(DecisionTreeRegressor(random_state=1))
# mdl_dt_cv = DecisionTreeRegressor()
depths = np.arange(1, 21)
num_leafs = [1, 5, 10, 20, 50, 100]
param_grid = [{'decisiontreeregressor__max_depth': depths, 'decisiontreeregressor__min_samples_leaf': num_leafs}]

In [ ]:
gs_dt = GridSearchCV(estimator=pipe_tree, param_grid=param_grid, scoring=scorer, cv=10)

In [ ]:
gs_dt.fit(X, y)

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('decisiontreeregressor',
                                        DecisionTreeRegressor(ccp_alpha=0.0,
                                                              criterion='mse',
                                                              max_depth=None,
                                                              max_features=None,
                                                              max_leaf_nodes=None,
                                                              min_impurity_decrease=0.0,
                                                              min_impurity_split=None,
                                                              min_samples_leaf=1,
                                                              min_samples_split=2,
                                                              min_weight_fraction_leaf=0.0,
                      

In [ ]:
gs_dt.best_params_

{'decisiontreeregressor__max_depth': 9,
 'decisiontreeregressor__min_samples_leaf': 1}

In [ ]:
mdl_dt = DecisionTreeRegressor(criterion='mse', max_depth=9, min_samples_leaf=1).fit(X, y)
scores_dt = cross_validate(mdl_dt, X, y, scoring=('r2', 'neg_mean_squared_error'), cv=cv, n_jobs=-1)

In [ ]:
scores_dt

{'fit_time': array([0.11542535, 0.11813927, 0.1151669 , 0.11536932, 0.11186051,
        0.12660408, 0.11641216, 0.12375641, 0.11455798, 0.11247802]),
 'score_time': array([0.00445986, 0.0050931 , 0.00462508, 0.00481701, 0.00450373,
        0.0046618 , 0.00406623, 0.00478268, 0.00453115, 0.0029161 ]),
 'test_neg_mean_squared_error': array([-2.68907563e-05, -5.47368421e-06, -6.10526316e-06, -1.38947368e-05,
        -7.57894737e-06, -3.59157895e-04, -1.47368421e-06, -1.62573099e-06,
        -5.26315789e-06, -2.60365199e-06]),
 'test_r2': array([0.99951301, 0.99989919, 0.99989394, 0.99976096, 0.99986878,
        0.99389516, 0.99997054, 0.99997407, 0.99990575, 0.99995599])}

In [ ]:
kf_res['1_model'].append('decision_tree')
kf_res['2_fit_time'].append(np.mean(scores_dt['fit_time']))
kf_res['3_score_time'].append(np.mean(scores_dt['score_time']))
kf_res['4_r2'].append(np.mean(scores_dt['test_r2']))
kf_res['5_mse'].append(np.abs(np.mean(scores_dt['test_neg_mean_squared_error'])))

*Random Forest Regressor*

In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

mdl_rf_cv = RandomForestRegressor()

In [ ]:
gs_rf = GridSearchCV(estimator = mdl_rf_cv, param_grid = param_grid, cv = 10, n_jobs = -1, verbose = 2).fit(X, y)

Fitting 10 folds for each of 288 candidates, totalling 2880 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   20.4s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 1009 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 1454 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done 1981 tasks      | elapsed: 19.7min
[Parallel(n_jobs=-1)]: Done 2588 tasks      | elapsed: 25.6min
[Parallel(n_jobs=-1)]: Done 2880 out of 2880 | elapsed: 28.9min finished


In [ ]:
gs_rf.best_params_

{'bootstrap': True,
 'max_depth': 90,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 10,
 'n_estimators': 300}

In [ ]:
mdl_rf = RandomForestRegressor(n_estimators=100, criterion='mse', max_depth=110, 
                               min_samples_split=8, min_samples_leaf=3, max_features=3, bootstrap=True).fit(X, y)
scores_rf = cross_validate(mdl_rf, X, y, scoring=('r2', 'neg_mean_squared_error'), cv=cv, n_jobs=-1)

In [ ]:
scores_rf

{'fit_time': array([0.35858655, 0.35704684, 0.35040617, 0.37462163, 0.34792948,
        0.38491893, 0.34580922, 0.33575678, 0.33849812, 0.32069898]),
 'score_time': array([0.01783252, 0.01890182, 0.0173831 , 0.02149534, 0.01635337,
        0.01803255, 0.01680803, 0.01794195, 0.01675487, 0.01032591]),
 'test_neg_mean_squared_error': array([-0.04846817, -0.04810563, -0.05090337, -0.05392781, -0.05174194,
        -0.05190534, -0.04113575, -0.05661853, -0.04937027, -0.0534954 ]),
 'test_r2': array([0.12223738, 0.11405086, 0.115686  , 0.07225687, 0.10412193,
        0.11773162, 0.17758952, 0.09690901, 0.11587002, 0.09578496])}

In [ ]:
kf_res['1_model'].append('random_forest')
kf_res['2_fit_time'].append(np.mean(scores_rf['fit_time']))
kf_res['3_score_time'].append(np.mean(scores_rf['score_time']))
kf_res['4_r2'].append(np.mean(scores_rf['test_r2']))
kf_res['5_mse'].append(np.abs(np.mean(scores_rf['test_neg_mean_squared_error'])))

*Bayesian Ridge*

In [ ]:
from sklearn.linear_model import BayesianRidge

mdl_byr = BayesianRidge().fit(X, y)
scores_byr = cross_validate(mdl_byr, X, y, scoring=('r2', 'neg_mean_squared_error'), cv=cv, n_jobs=-1)

In [ ]:
scores_byr

{'fit_time': array([0.71164083, 0.68576789, 0.65133333, 0.66252971, 0.63752627,
        0.66907549, 0.64924979, 0.65121961, 0.63149786, 0.61263585]),
 'score_time': array([0.00401711, 0.00421357, 0.004143  , 0.00395489, 0.00453401,
        0.00374913, 0.0047524 , 0.00372386, 0.0043726 , 0.00435352]),
 'test_neg_mean_squared_error': array([-1.41561003e-17, -9.94818824e-18, -6.15342569e-17, -3.09067439e-18,
        -2.41529241e-18, -8.09448764e-17, -1.62063551e-17, -7.60254604e-17,
        -3.38886572e-17, -2.60084360e-17]),
 'test_r2': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

In [ ]:
kf_res['1_model'].append('bayesian ridge')
kf_res['2_fit_time'].append(np.mean(scores_byr['fit_time']))
kf_res['3_score_time'].append(np.mean(scores_byr['score_time']))
kf_res['4_r2'].append(np.mean(scores_byr['test_r2']))
kf_res['5_mse'].append(np.abs(np.mean(scores_byr['test_neg_mean_squared_error'])))

In [ ]:
kf_res

{'1_model': ['linear regression',
  'ridge',
  'lasso',
  'elastic_net',
  'decision_tree',
  'random_forest',
  'bayesian ridge'],
 '2_fit_time': [0.2964270353317261,
  0.10867724418640137,
  0.047791481018066406,
  0.04957842826843262,
  0.11697700023651122,
  0.3514272689819336,
  0.6562476634979248],
 '3_score_time': [0.0047044515609741214,
  0.004502630233764649,
  0.004514479637145996,
  0.004511117935180664,
  0.004445672035217285,
  0.01718294620513916,
  0.004181408882141113],
 '4_r2': [1.0,
  0.9999997203937573,
  -0.0017588871356867264,
  -0.0017588866246517787,
  0.9992637380667297,
  0.11322381533150867,
  0.9999999999999994],
 '5_mse': [1.229506461893756e-20,
  1.595645178552409e-08,
  0.05705381678134205,
  0.057053816751893416,
  4.300675077048357e-05,
  0.05056722014457775,
  3.242182973879564e-17]}

*Results*

In [51]:
kf_results = pd.DataFrame(kf_res)

In [52]:
kf_results

,1_model,2_fit_time,3_score_time,4_r2,5_mse
0,linear regression,0.296427,0.004704,1.000000,1.229506e-20
1,ridge,0.108677,0.004503,1.000000,1.595645e-08
2,lasso,0.047791,0.004514,-0.001759,5.705382e-02
3,elastic_net,0.049578,0.004511,-0.001759,5.705382e-02
4,decision_tree,0.116977,0.004446,0.999264,4.300675e-05
5,random_forest,0.351427,0.017183,0.113224,5.056722e-02
6,bayesian ridge,0.656248,0.004181,1.000000,3.242183e-17
